In [16]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import InputLayer
from keras.layers.advanced_activations import LeakyReLU
import json
from PIL import Image
from scipy import misc
import numpy as np
import os



In [17]:
model = load_model('C:/Users/CEO/기복/weight/epoch 200.h5')## 가중치를 불러옵니다. (트레이닝한 가중치가 이미 존재한다는 전제하입니다. 아니라면 트레이닝으로 바로 가셔도 됩니다.)
def predict(image_path):
    
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('C:/Users/CEO/Downloads/DBSRCNN/DBSRCNN/source/{}'.format(file))
    
    ref = modcrop(ref, 3)
    degraded = modcrop(degraded, 3)
    
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    Y = numpy.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    pre = srcnn.predict(Y, batch_size=1)
    
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    temp = shave(temp, 6)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)

In [18]:
def model():
    
    SRCNN = Sequential()
    
    SRCNN.add(Conv2D(filters=128, kernel_size = (9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    SRCNN.add(Conv2D(filters=64, kernel_size = (3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=1, kernel_size = (5, 5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
    adam = Adam(lr=0.0003)
    
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])

In [21]:
for file in os.listdir('C:/Users/CEO/기복/input'):
    
    ref, degraded, output, scores = predict('C:/Users/CEO/기복/output/{}'.format(file))